In [1]:
import pandas as pd

### Load Data

In [2]:
years = ['2018','2019','2020','2021']
demands = [pd.read_excel(open('demands.xls', 'rb'), sheet_name=year) for year in years]

### Pre Processing

In [3]:
for i in range(len(years)):
     demands[i]["YEAR"]=int(years[i])
total_demands = pd.concat(demands)

In [4]:
total_demands

,SKU_ID,'01','02','03','04','05','06','07','08','09','10','11','12',TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,1652.0,1297.0,1285.0,19228,1602.3,150.0,432,408,2018
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,15865.0,17097.0,17987.0,145938,12161.5,1152.0,2304,1104,2018
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,NaN,92.0,205.0,180.0,168.0,2551,212.6,24.0,48,0,2018
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,948.0,3504.0,1026.0,10536,878.0,144.0,240,126,2018
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,40.0,29.0,99.0,493,41.1,60.0,120,127,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,NaN,NaN,NaN,NaN,NaN,NaN,12.0,99.0,21.0,39.0,24.0,23.0,218,18.2,56.0,112,60,2021
3008,WT420409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3,0.3,20.0,40,47,2021
3009,SQ336060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,44,3.7,28.0,56,30,2021
3010,0SK00441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,17,1.4,240.0,480,259,2021


In [5]:
assuta_items = pd.read_excel("Assuta_items.xlsx")
assuta_items

,SKU_ID,Is_Assuta
0,AS100016,Y
1,AS120257,Y
2,AS110041,Y
3,AS180293,Y
4,AS040107,Y
...,...,...
1285,AS400185,Y
1286,AS010122,Y
1287,AS000201,Y
1288,ASA00006,Y


In [6]:
total_demands = total_demands[~total_demands["SKU_ID"].isin(assuta_items["SKU_ID"])].copy()

In [7]:
total_demands.fillna(0,inplace=True)

In [8]:
total_demands

,SKU_ID,'01','02','03','04','05','06','07','08','09','10','11','12',TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,1652.0,1297.0,1285.0,19228,1602.3,150.0,432,408,2018
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,15865.0,17097.0,17987.0,145938,12161.5,1152.0,2304,1104,2018
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,0.0,92.0,205.0,180.0,168.0,2551,212.6,24.0,48,0,2018
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,948.0,3504.0,1026.0,10536,878.0,144.0,240,126,2018
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,40.0,29.0,99.0,493,41.1,60.0,120,127,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,0.0,0.0,0.0,0.0,0.0,0.0,12.0,99.0,21.0,39.0,24.0,23.0,218,18.2,56.0,112,60,2021
3008,WT420409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3,0.3,20.0,40,47,2021
3009,SQ336060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,44,3.7,28.0,56,30,2021
3010,0SK00441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,17,1.4,240.0,480,259,2021


In [9]:
# Add quarters columns
total_demands["Q1"] = total_demands["'01'"] + total_demands["'02'"] + total_demands["'03'"]
total_demands["Q2"] = total_demands["'04'"] + total_demands["'05'"] + total_demands["'06'"]
total_demands["Q3"] = total_demands["'07'"] + total_demands["'08'"] + total_demands["'09'"]
total_demands["Q4"] = total_demands["'10'"] + total_demands["'11'"] + total_demands["'12'"]

In [10]:
total_demands

,SKU_ID,'01','02','03','04','05','06','07','08','09',...,TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR,Q1,Q2,Q3,Q4
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,...,19228,1602.3,150.0,432,408,2018,5991.0,4295.0,4708.0,4234.0
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,...,145938,12161.5,1152.0,2304,1104,2018,34064.0,35444.0,25481.0,50949.0
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,0.0,92.0,...,2551,212.6,24.0,48,0,2018,1099.0,681.0,218.0,553.0
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,...,10536,878.0,144.0,240,126,2018,2268.0,1218.0,1572.0,5478.0
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,...,493,41.1,60.0,120,127,2018,154.0,82.0,89.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,0.0,0.0,0.0,0.0,0.0,0.0,12.0,99.0,21.0,...,218,18.2,56.0,112,60,2021,0.0,0.0,132.0,86.0
3008,WT420409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,0.3,20.0,40,47,2021,0.0,0.0,0.0,3.0
3009,SQ336060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44,3.7,28.0,56,30,2021,0.0,0.0,0.0,44.0
3010,0SK00441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17,1.4,240.0,480,259,2021,0.0,0.0,0.0,17.0


### Winters Model

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline

In [47]:
def seasonality (demand_series,seasonal_periods):
    fit = ExponentialSmoothing(demand_series,seasonal_periods=seasonal_periods,trend="add",seasonal="mul",use_boxcox=True,initialization_method="estimated").fit()
    # fit = ExponentialSmoothing(demand_series,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,use_boxcox=True,initialization_method="estimated").fit()
    results = pd.DataFrame(index=[r"$\alpha$", r"$\beta$", r"$\phi$", r"$\gamma$", r"$l_0$", "$b_0$", "SSE"])   
    params = ["smoothing_level","smoothing_trend","damping_trend","smoothing_seasonal","initial_level","initial_trend"]
    results["Multiplica Dam"] = [fit.params[p] for p in params] + [fit.sse]
    # print(results)
    fit = ExponentialSmoothing(demand_series,seasonal_periods=seasonal_periods,trend="add",seasonal="mul",initialization_method="estimated").fit()
    # fit = ExponentialSmoothing(demand_series,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,initialization_method="estimated").fit()
    df = pd.DataFrame(np.c_[aust, fit.level, fit.trend, fit.season, fit.fittedvalues], columns=['Y_t', 'L_t', 'B_t', 'S_t', 'Y_t_hat'],index=aust.index)
    # df.append(fit.forecast(8).rename(r"$/hat{y}_t$").to_frame(), sort=True)
    # a = pd.concat(df,df, ignore_index=True)
    return pd.concat([df['S_t'].iloc[:seasonal_periods],df['B_t'].iloc[:1]]).to_numpy()

data = [2,3,10,11,1,4,11,10,3,2]
index = pd.date_range(start="2005", end="2015", freq="Y")
aust = pd.Series(data, index)
seasonality(aust,4)

array([ 0.31802782,  0.47894228,  1.67139485,  1.67519218, -0.01192725])

In [71]:
unique_sku_ids = total_demands['SKU_ID'].unique()
for i in unique_sku_ids:
    a = total_demands[total_demands['SKU_ID'] == i]
    
    if i == 'NU802536':
        print(a)

    
unique_sku_ids
# pd.nunique(total_demands['SKU_ID']).shape
# total_demands['SKU_ID'].value_counts().to_frame()
# 2898 X 4 = 11592
# 9084

        SKU_ID    '01'    '02'    '03'    '04'    '05'    '06'    '07'  \
0     NU802536  2839.0  1513.0  1639.0  1466.0  1484.0  1345.0  1812.0   
12    NU802536  1398.0  1295.0  1748.0  1481.0  1212.0  1224.0  1162.0   
9     NU802536  1381.0  1112.0   625.0   122.0  1319.0   883.0   710.0   
1798  NU802536   758.0   732.0   785.0   934.0   748.0   806.0   691.0   

        '08'    '09'  ...  TOTAL_FOR_YEAR  AVG_PER_MONTH  TRIGGER_QTY  \
0     1271.0  1625.0  ...           19228         1602.3        150.0   
12    1190.0  1184.0  ...           15966         1330.5        150.0   
9      860.0   771.0  ...           10265          855.4        150.0   
1798   965.0   751.0  ...            9072          756.0        150.0   

      MAXIMUM_QTY  QTY_ON_HAND  YEAR      Q1      Q2      Q3      Q4  
0             432          408  2018  5991.0  4295.0  4708.0  4234.0  
12            432          408  2019  4441.0  3917.0  3536.0  4072.0  
9             432          408  2020  3118.0  2324

array(['NU802536', 14571184, 'RB038125', ..., 'SQ336060', '0SK00441',
       'JC577640'], dtype=object)

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline

def seasonality (demand_series):
    # fit = ExponentialSmoothing(demand_series,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,use_boxcox=True,initialization_method="estimated").fit()
    fit = ExponentialSmoothing(demand_series,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,use_boxcox=True,initialization_method="estimated").fit()
    results = pd.DataFrame(index=[r"$\alpha$", r"$\beta$", r"$\phi$", r"$\gamma$", r"$l_0$", "$b_0$", "SSE"])   
    params = ["smoothing_level","smoothing_trend","damping_trend","smoothing_seasonal","initial_level","initial_trend"]
    results["Multiplica Dam"] = [fit.params[p] for p in params] + [fit.sse]
    # print(results)
    # fit = ExponentialSmoothing(demand_series,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,initialization_method="estimated").fit()
    fit = ExponentialSmoothing(demand_series,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,initialization_method="estimated").fit()
    df = pd.DataFrame(np.c_[aust, fit.level, fit.trend, fit.season, fit.fittedvalues], columns=[r"$y_t$", r"$l_t$", r"$b_t$", r"$s_t$", r"$\hat{y}_t$"],index=aust.index,)
    df.append(fit.forecast(8).rename(r"$\hat{y}_t$").to_frame(), sort=True)
 
    

data = [446.6565,454.4733,455.663,423.6322,456.2713,440.5881,425.3325,485.1494,506.0482,526.792,514.2689,494.211,]
index = pd.date_range(start="1996", end="2008", freq="A")
oildata = pd.Series(data, index)

data = [17.5534,21.86,23.8866,26.9293,26.8885,28.8314,30.0751,30.9535,30.1857,31.5797,32.5776,33.4774,39.0216,41.3864,41.5966,]
index = pd.date_range(start="1990", end="2005", freq="A")
air = pd.Series(data, index)

data = [263.9177,268.3072,260.6626,266.6394,277.5158,283.834,290.309,292.4742,300.8307,309.2867,318.3311,329.3724,338.884,339.2441,328.6006,314.2554,
    314.4597,321.4138,329.7893,346.3852,352.2979,348.3705,417.5629,417.1236,417.7495,412.2339,411.9468,394.6971,401.4993,408.2705,414.2428,]
index = pd.date_range(start="1970", end="2001", freq="A")
livestock2 = pd.Series(data, index)

data = [407.9979, 403.4608, 413.8249, 428.105, 445.3387, 452.9942, 455.7402]
index = pd.date_range(start="2001", end="2008", freq="A")
livestock3 = pd.Series(data, index)

# data = [41.7275,24.0418,32.3281,37.3287,46.2132,29.3463,36.4829,42.9777,48.9015,31.1802,37.7179,40.4202,51.2069,31.8872,40.9783,43.7725,
#     55.5586,33.8509,42.0764,45.6423,59.7668,35.1919,44.3197,47.9137,]
# data = [5.5,5.625,5.5,9.05,5.175,5.4,4.95,8.25,4.775,4.825]
data = [2,3,10,11,1,4,11,10,3,2]
# index = pd.date_range(start="2005", end="2010-Q4", freq="QS-OCT")
index = pd.date_range(start="2005", end="2015", freq="Y")
aust = pd.Series(data, index)
print(aust)

2005-12-31     2
2006-12-31     3
2007-12-31    10
2008-12-31    11
2009-12-31     1
2010-12-31     4
2011-12-31    11
2012-12-31    10
2013-12-31     3
2014-12-31     2
Freq: A-DEC, dtype: int64


In [14]:
aust = np.array([2,3,10,11,1,4,11,10,3,2])
fit1 = ExponentialSmoothing(aust,seasonal_periods=4,trend="add",seasonal="add",use_boxcox=True,initialization_method="estimated",).fit()
fit2 = ExponentialSmoothing(aust,seasonal_periods=4,trend="add",seasonal="mul",use_boxcox=True,initialization_method="estimated",).fit()
fit3 = ExponentialSmoothing(aust,seasonal_periods=4,trend="add",seasonal="add",damped_trend=True,use_boxcox=True,initialization_method="estimated",).fit()
fit4 = ExponentialSmoothing(aust,seasonal_periods=4,trend="add",seasonal="mul",damped_trend=True,use_boxcox=True,initialization_method="estimated",).fit()
results = pd.DataFrame(index=[r"$/alpha$", r"$/beta$", r"$/phi$", r"$/gamma$", r"$l_0$", "$b_0$", "SSE"])
params = ["smoothing_level","smoothing_trend","damping_trend","smoothing_seasonal","initial_level","initial_trend",]
results["Additive"] = [fit1.params[p] for p in params] + [fit1.sse]
results["Multiplicative"] = [fit2.params[p] for p in params] + [fit2.sse]
results["Additive Dam"] = [fit3.params[p] for p in params] + [fit3.sse]
results["Multiplica Dam"] = [fit4.params[p] for p in params] + [fit4.sse]

# ax = aust.plot(figsize=(10, 6),marker="o",color="black",title="Forecasts from Holt-Winters' multiplicative method",)
# ax.set_ylabel("International visitor night in Australia (millions)")
# ax.set_xlabel("Year")
# fit1.fittedvalues.plot(ax=ax, style="--", color="red")
# fit2.fittedvalues.plot(ax=ax, style="--", color="green")

# fit1.forecast(8).rename("Holt-Winters (add-add-seasonal)").plot(ax=ax, style="--", marker="o", color="red", legend=True)
# fit2.forecast(8).rename("Holt-Winters (add-mul-seasonal)").plot(ax=ax, style="--", marker="o", color="green", legend=True)

# plt.show()
# print("Figure 7.6: Forecasting international visitor nights in Australia using Holt-Winters method with both additive and multiplicative seasonality.")

results

,Additive,Multiplicative,Additive Dam,Multiplica Dam
$/alpha$,1.490116e-08,1.490116e-08,1.490116e-08,1.490116e-08
$/beta$,1.490083e-08,2.976556e-10,1.490083e-08,2.976391e-10
$/phi$,NaN,NaN,9.856090e-01,9.950000e-01
$/gamma$,0.000000e+00,2.439094e-16,4.672103e-17,0.000000e+00
$l_0$,1.813135e+00,1.856335e+00,1.813235e+00,1.857466e+00
$b_0$,1.416478e-07,-9.872587e-03,-1.214876e-05,-1.001697e-02
SSE,5.107737e+00,5.254750e+00,5.107550e+00,5.252240e+00


In [15]:
# %pip install statsmodels
# import statsmodels.api
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [16]:
# Set the value of Alpha and define m (Time Period)
m = 12
alpha = 1/(2*m)
a=[5.5,5.625,5.5,9.05,5.175,5.4,4.95,8.25,4.775,4.825]
# b = ExponentialSmoothing(a,trend='mul',seasonal='mul',seasonal_periods=4,initialization_method="estimated").fit()
# # initial_values(a)
# print(ExponentialSmoothing.initial_values(b))

In [17]:
fit = ExponentialSmoothing(a, seasonal_periods=4, trend='mul', seasonal='mul', initialization_method="estimated").fit()
simulations = fit.simulate(5, repetitions=100, error='mul')
fit.fittedvalues.plot(ax=ax, style='--', color='green')
simulations.plot(ax=ax, style='-', alpha=0.05, color='grey', legend=False)
fit.forecast(8).rename('Holt-Winters (add-mul-seasonal)').plot(ax=ax, style='--', marker='o', color='green', legend=True)

C:\Users\omery\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'plot'

In [ ]:
airline[‘HWES3_MUL’] = ExponentialSmoothing(airline[‘Thousands of Passengers’],trend=’mul’,seasonal=’mul’,
                                            seasonal_periods=12).fit().
fittedvalues
airline[[‘Thousands of Passengers’,’HWES3_ADD’,’HWES3_MUL’]].plot(title=’Holt Winters Triple Exponential Smoothing: Additive and Multiplicative Seasonality’);

In [ ]:
# a=list(total_demands["SKU_ID"])
# for i in a:
#     if "AS" in str(i):
#         print(i)

In [ ]:
# Group A - find the ~20% with highest demands
# for every product, calculate the average demand for the 4 years
annual_avg = ((total_demands.groupby("SKU_ID")["TOTAL_FOR_YEAR"].sum())/4).to_frame()
annual_avg.columns = ['YEARLY_AVG']
annual_avg.reset_index(inplace=True)
# annual_avg.head()

In [ ]:
annual_avg